# ... project description ...
https://github.com/comp-data/2021-2022/tree/main/docs/project

# What is done for Relational Database and Queries:

### Creating Database publications.db and Tables 
#### with CSV relational_publications.csv
#### with JSON relational_other_data.json

In [2]:
from sqlite3 import connect

with connect("publications6.db") as con:
    con.commit()  
    cursor = con.cursor()

### CSV...

'''
csv_file = open("./data/relational_publications.csv", encoding='utf-8')
contents = csv.reader(csv_file)
'''

In [38]:
import pandas as pd
from pandas import read_csv, Series
import csv

csv_file = open("./data/relational_publications.csv", encoding="utf-8")
contents = csv.reader(csv_file)

csv_file = pd.read_csv("./data/relational_publications.csv", keep_default_na=False,
                        dtype={
                                    "id": "string",
                                    "title": "string",
                                    "type": "string",
                                    "publication_year": "string",
                                    "issue": "string",
                                    "volume": "string",
                                    "chapter": "string",
                                    "publication_venue": "string",
                                    "venue_type": "string",
                                    "publisher": "string",
                                    "event": "string"

                        },encoding="utf-8")

# Create a new column with internal identifiers for each publication
#csvfile_internal_id = []
#for idx, row in csv_file.iterrows():
#    csvfile_internal_id.append("publication-" + str(idx))
#csv_file.insert(0, "internalId", Series(csvfile_internal_id, dtype="string"))
csv_file

,id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event
0,doi:10.1162/qss_a_00023,"Opencitations, An Infrastructure Organization ...",journal-article,2020,1,1,,Quantitative Science Studies,journal,crossref:281,
1,doi:10.1007/s11192-019-03217-6,"Software Review: Coci, The Opencitations Index...",journal-article,2019,2,121,,Scientometrics,journal,crossref:297,
2,doi:10.1007/s11192-019-03311-9,Nine Million Book Items And Eleven Million Cit...,journal-article,2019,2,122,,Scientometrics,journal,crossref:297,
3,doi:10.1038/sdata.2016.18,The Fair Guiding Principles For Scientific Dat...,journal-article,2016,1,3,,Scientific Data,journal,crossref:297,
4,doi:10.1371/journal.pbio.3000385,The Nih Open Citation Collection: A Public Acc...,journal-article,2019,10,17,,Plos Biology,journal,crossref:340,
...,...,...,...,...,...,...,...,...,...,...,...
495,doi:10.3390/admsci10030069,Performance Analysis And Science Mapping Of In...,journal-article,2020,3,10,,Administrative Sciences,journal,crossref:1968,
496,doi:10.1186/s12888-020-02825-4,Mapping The Literature On Parents With Mental ...,journal-article,2020,1,20,,Bmc Psychiatry,journal,crossref:297,
497,doi:10.1080/00472778.2020.1776578,Evolution Of The Entrepreneurship And Innovati...,journal-article,2020,,,,Journal Of Small Business Management,journal,crossref:301,
498,doi:10.1007/s11301-020-00196-4,Intellectual Structure Of Management Innovatio...,journal-article,2020,3,71,,Management Review Quarterly,journal,crossref:297,


In [39]:
create_csvfile = '''
DROP TABLE IF EXISTS cvsfile;
CREATE TABLE IF NOT EXISTS csvfile
               (id, title, type, publication_year ,issue, volume, chapter, publication_venue, venue_type, publisher,event
              );
              '''

In [40]:
cursor.executescript(create_csvfile)

In [41]:

insert_records = "INSERT INTO csvfile (id, title, type, publication_year, issue, volume, chapter, publication_venue, venue_type, publisher, event) VALUES(?,?,?,?,?,?,?,?,?,?,?)"

cursor.executemany(insert_records, contents)

In [42]:
cursor.executescript(create_csvfile)

#### ... finished with CSV ...

### Creating tables from JSON

In [4]:
from json import load
import json
import pandas as pd

# importing authors from JSON

from collections import deque

with open("./data/relational_other_data.json", "r", encoding="utf-8") as f:
    json_doc = load(f)
#print(type(json_doc))

# read just authors

authors = json_doc["authors"]

# create the dataframe from json

df_authors=pd.DataFrame(authors.items(),columns=['doi','author']).explode('author')

df_authors=pd.json_normalize(json.loads(df_authors.to_json(orient="records")))
df_authors.rename(columns={"author.family":"family_name","author.given":"given_name","author.orcid":"orc_id"}, inplace = True)

df_authors=pd.DataFrame(df_authors)

In [3]:
cursor = con.cursor()
from pandasql import sqldf

def pysqldf(q):
    """this function eliminates the need to include locals/globals all the time"""
    return sqldf(q, globals())

NameError: name 'con' is not defined

#### Authors from JSON

In [45]:
create_authors = '''
DROP TABLE IF EXISTS authors;

CREATE TABLE IF NOT EXISTS authors (
    orc_id      TEXT PRIMARY KEY,
    doi         TEXT,
    family_name TEXT,
    given_name  TEXT

);
'''

In [46]:
cursor.executescript(create_authors)

In [47]:
pysqldf('''select doi ,
         orc_id, family_name,
         given_name from df_authors;''').to_sql('authors',
                                               con, index=False,
                                               if_exists='replace')

1138

#### Publisher from JSON

In [48]:
### importing publisher JSON

# importing publisher JSON

# read just publishers

publisher = json_doc["publishers"]

# create the dataframe from json

df_publishers = pd.DataFrame(publisher.items(), columns=['crossref', 'name'])

df_publishers = pd.json_normalize(json.loads(df_publishers.to_json(orient="records")))
df_publishers.rename(columns={"name.id": "id", "name.name": "name"}, inplace=True)

In [49]:
create_publishers = '''
DROP TABLE IF EXISTS publishers;

CREATE TABLE IF NOT EXISTS publishers (
    crossref    TEXT PRIMARY KEY,
    name TEXT
   );
'''

In [50]:
cursor.executescript(create_publishers)

In [51]:
pysqldf('''select crossref ,
           name from df_publishers;''').to_sql('publishers',
                                               con, index=False,
                                               if_exists='replace')

32

#### References from JSON

In [52]:
references = json_doc["references"]

# create the dataframe from json

df_references = pd.DataFrame(references.items(), columns=['doi', 'refid']).explode('refid')

df_references = pd.json_normalize(json.loads(df_references.to_json(orient="records")))

#df_publishers.rename(columns={"name.id": "id", "name.name": "name"}, inplace=True)

In [55]:
create_references = '''
DROP TABLE IF EXISTS references_json;

CREATE TABLE IF NOT EXISTS references_json (
    doi    TEXT,
    refid TEXT
   );
'''

In [56]:
cursor.executescript(create_references)


In [57]:
pysqldf('''select doi ,
           refid from df_references;''').to_sql('references_json',
                                               con, index=False,
                                               if_exists='replace')

567

In [6]:
query16="SELECT doi,COUNT(*) FROM references_json GROUP BY doi"
result16=cursor.execute(query16).fetchall()

result16

[('doi:10.1002/adom.202100519', 1),
 ('doi:10.1002/agj2.20628', 1),
 ('doi:10.1002/asi.23267', 1),
 ('doi:10.1002/asi.23630', 1),
 ('doi:10.1002/asi.23770', 1),
 ('doi:10.1002/asi.24130', 1),
 ('doi:10.1002/asi.24171', 1),
 ('doi:10.1002/asi.24301', 1),
 ('doi:10.1002/bbb.2290', 1),
 ('doi:10.1002/cjce.23344', 1),
 ('doi:10.1002/cjce.23346', 1),
 ('doi:10.1002/cjce.23466', 1),
 ('doi:10.1002/cjce.23530', 1),
 ('doi:10.1002/cjce.23913', 1),
 ('doi:10.1002/cjce.24089', 1),
 ('doi:10.1002/cjce.24127', 1),
 ('doi:10.1002/cjce.24216', 1),
 ('doi:10.1002/csr.1792', 1),
 ('doi:10.1002/ece3.7980', 1),
 ('doi:10.1002/hfm.20889', 1),
 ('doi:10.1002/jclp.23227', 1),
 ('doi:10.1002/kpm.1658', 1),
 ('doi:10.1002/leap.1114', 1),
 ('doi:10.1002/mabi.202100103', 1),
 ('doi:10.1002/pri.1760', 1),
 ('doi:10.1002/rra.3431', 1),
 ('doi:10.1002/sd.1927', 1),
 ('doi:10.1002/sdr.1628', 1),
 ('doi:10.1002/sres.2731', 1),
 ('doi:10.1007/978-3-030-16187-3_20', 1),
 ('doi:10.1007/978-3-030-53036-5_30', 1),
 ('do

#### Venues id from JSON

In [ ]:
venues_idjson = json_doc["venues_id"]

# create the dataframe from json

df_venuesjson=pd.DataFrame(venues_idjson.items(),columns=['doi','issn']).explode('issn')

df_venuesjson=pd.json_normalize(json.loads(df_venuesjson.to_json(orient="records")))
#df_venues.rename(columns={"author.family":"family_name","author.given":"given_name","author.orcid":"orc_id"}, inplace = True)

df_venuesjson=pd.DataFrame(df_venuesjson)

In [ ]:
create_venuesjson = '''
DROP TABLE IF EXISTS venuesid_json;

CREATE TABLE IF NOT EXISTS venuesid_json (
    doi    TEXT,
    issn TEXT PRIMARY KEY
   );
'''

In [ ]:
cursor.executescript(create_venuesjson)

In [ ]:
pysqldf('''select doi ,
           issn from df_venuesjson;''').to_sql('venuesid_json',
                                                con, index=False,
                                                if_exists='replace')

# ... finished with tables ....

#### Attempting creating methods that project requires.
#### They are not Class now... Only functionality

In [ ]:
#getProceedingsByEvent --> deve diventare il metodo della sua classe

con.commit()
query= "SELECT * FROM csvfile WHERE publication_venue LIKE ? COLLATE NOCASE"
some_query_value = '%web%'
result= cursor.execute(query,[some_query_value]).fetchall()
df_getProceedingsByEvent = pd.DataFrame(result)

df_getProceedingsByEvent

In [ ]:
#getPublicationsPublishedInYear --> by the way it should be programmed as method 
#in order to use in Class

query1= "SELECT * FROM csvfile WHERE publication_year= '2020'"
result1= cursor.execute(query1).fetchall()
df_getPublicationsPublishedInYear = pd.DataFrame(result1)

#df_getPublicationsPublishedInYear

In [ ]:
#getPublicationsByAuthorId --> deve diventare il metodo della sua classe

query3= "SELECT* FROM authors WHERE orc_id='0000-0001-9857-1511'"

result=cursor.execute(query3).fetchall()
df_getPublicationsByAuthorId = pd.DataFrame(result)

df_getPublicationsByAuthorId

In [ ]:
#getPublicationsByAuthorName --> deve diventare il metodo della sua classe
query7= "SELECT* FROM authors WHERE given_name LIKE ? COLLATE NOCASE"
value= '%hara%'
result7= cursor.execute(query7,[value]).fetchall()
df_getPublicationsByAuthorName = pd.DataFrame(result7)

df_getPublicationsByAuthorName

In [ ]:
#getVenuesByPublisherId --> deve diventare il metodo della sua classe

query4= "SELECT* FROM publishers WHERE crossref='crossref:78'"
result4=cursor.execute(query4).fetchall()
df_getVenuesByPublisherId= pd.DataFrame(result4)

df_getVenuesByPublisherId

In [ ]:
#getPublicationInVenue --> deve diventare il metodo della sua classe
query5= "SELECT* FROM venuesid_json WHERE issn='issn:0944-1344' "
result5=cursor.execute(query5).fetchall()
df_getPublicationInVenue= pd.DataFrame(result5)

df_getPublicationInVenue

In [ ]:
#getJournalArticlesInIssue --> by the way it should be programmed as method 
#in order to use in Class

query8= "SELECT doi FROM venuesid_json WHERE issn='issn:2164-5515'"
result8=cursor.execute(query8).fetchall()

for element in result8:
  query9= "SELECT* FROM csvfile WHERE type='journal-article' AND issue='9' AND volume='17' AND id=? "
  result9=cursor.execute(query9, element).fetchall()
df_getJournalArticlesInIssue= pd.DataFrame(result9)

df_getJournalArticlesInIssue

In [ ]:
#getJournalArticlesInVolume --> by the way it should be programmed as method 
#in order to use in Class

query10= "SELECT doi FROM venuesid_json WHERE issn='issn:2164-5515'"
result10=cursor.execute(query10).fetchall()

for element in result10:
  query11= "SELECT* FROM csvfile WHERE type='journal-article' AND volume='17' AND id=? "
  result11=cursor.execute(query11, element).fetchall()
df_getJournalArticlesInVolume= pd.DataFrame(result11)

df_getJournalArticlesInVolume

#### some problems with implementing these:

##### getDistinctPublisherOfPublications
##### getDistinctPublisherOfPublications
##### getMostCitedPublication
####  getMostCitedVenue

